---
title: How to submit a PwBaseWorkChain 
subtitle: 
#author:
#  - name: Miki Bonacci
    #affiliations: Executable Books; Curvenote
    #orcid: 0000-0002-7859-8394
#    email: miki.bonacci@psi.ch
license:
  code: MIT
#date: 2023/01/23
---

In the following we show how to run a calculation with the `aiida-quantumespresso` plugin using the new *atomistic* `StructureData`. 
The procedure is very similar to the [old way](https://aiida-quantumespresso.readthedocs.io/en/latest/tutorials/first_pw.html), except to the fact that now properties like magnetization and charge are provided within the structure
and not in the input parameters of the `PwCalculation`.

***For now, only magnetization is supported.***

***PLEASE USE THE FOLLOWING VERSIONS OF THE RELATED CODES:***

- aiida-atomistic: [switch_models branch](https://github.com/mikibonacci/aiida-atomistic/tree/switch_models)
- aiida-core: [new_StructureData branch](https://github.com/mikibonacci/aiida-core/tree/new_StructureData)
- aiida-quantumespresso: [new_StructureData branch](https://github.com/mikibonacci/aiida-quantumespresso/tree/new_StructureData)
- aiida-pseudo: [new_StructureData branch](https://github.com/mikibonacci/aiida-pseudo/tree/new_StructureData)

In [1]:
from aiida import load_profile, orm
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_atomistic import StructureData, StructureDataMutable

load_profile()

Profile<uuid='1a5a8d0836814a04a238c67cc7481655' name='default'>

Once loaded the necessary modules, we start building up our `StructureDataMutable`, to be then converted into the AiiDA `StructureData` (via the `to_immutable` method):

In [2]:
mutable_structure = StructureDataMutable()
mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

mutable_structure.add_atom(**{
            'symbols':'Si',
            'positions':[1/2, 1/2, 1/2],
            'kinds': 'Si1'
        })
mutable_structure.add_atom(**{
            'symbols':'Si',
            'positions':[3/4, 3/4, 3/4],
            'kinds': 'Si1'
        })

structure = mutable_structure.to_immutable()

We then generate the `PwBaseWorkChain` builder via the `get_builder_from_protocol` method, providing as input the *pw.x* code node and the structure:

In [3]:
from aiida.engine import submit, run_get_node


builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/26/2024 04:34:23 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1941|PwBaseWorkChain|run_process]: launching PwCalculation<1946> iteration #1
09/26/2024 04:34:27 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1941|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/26/2024 04:34:27 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1941|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


After the job is finished, you can inspect the outputs as usual:

In [4]:
run[0]

{'remote_folder': <RemoteData: uuid: 88fdebdf-0fae-4e0a-be77-f10a67a5bbfd (pk: 1947)>,
 'retrieved': <FolderData: uuid: 18958707-301f-492a-a921-50a70e3f9bbe (pk: 1948)>,
 'output_parameters': <Dict: uuid: df3d21f0-db14-48dd-97d4-6ecba66a2223 (pk: 1951)>,
 'output_trajectory': <TrajectoryData: uuid: 7d02ea97-6fb8-4946-893d-cdd3e7802239 (pk: 1950)>,
 'output_band': <BandsData: uuid: 9df14e07-cff5-4ba5-80ad-4a4cda4a4951 (pk: 1949)>}

## How to deal with magnetic configurations

The first way to prepare a magnetic calculation is to trigger the `nspin` or `noncolin` in the input parameters of the `PwCalculation` respectively for *collinear* (`nspin=2`) and *non collinear* (`noncolin=.True.`) calculations.

Of course, the magnetic moments have to be provided within the `StructureData`, so that they can be used to build the [*starting_magnetization*](https://www.quantum-espresso.org/Doc/INPUT_PW.html#idm301) variable in the `&SYSTEM` namelist.


### Collinear case 

We first load the structure using Pymatgen to parse the *mcif* file for BCC Iron:

In [5]:
from pymatgen.core import Structure

#suppose you are in the `aiida-atomistic` root folder
iron_bcc = Structure.from_file('../examples/structure/data/Fe_bcc.mcif', primitive=True)
magnetic_structure = StructureData.from_pymatgen(iron_bcc)

print(magnetic_structure.properties.magmoms)

[[0.0, 0.0, 2.5]]


In [6]:
magnetic_structure.properties.kinds

['Fe0']

We can check that our structure is collinear (trivial in this case):

In [7]:
magnetic_structure.is_collinear

True

For now, we consider only collinear magnetic moments along the z axis.

We can create again the builder as before, this time overriding `nspin` to be equal to 2 (*collinear* calculation):

In [8]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"nspin": 2}
            }
        }
    }
    )

In [9]:
run = run_get_node(builder)

09/26/2024 04:34:30 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1958|PwBaseWorkChain|run_process]: launching PwCalculation<1963> iteration #1
09/26/2024 04:34:39 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1958|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/26/2024 04:34:39 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1958|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


### The output magnetic moments

For now, the new output magnetization of a structure will not be stored in the output `StructureData` (if any). 
The reason is that the output magnetization can present some discrepancies which are actually artifacts of the simulation. For example, (magnetic) symmetries maybe not correctly detected (due to different kinds) and so magnetization magnitudes (and orientations) are not the same as expected. 

So, the if the user needs it, he can use a `calcfuntion` to generate a new `StructureData` with the new magnetic moments (both as list of *floats* or of len=3 *lists*):

In [10]:
trajectory = run[-1].called[-1].outputs.output_trajectory # run[1].called[-1].outputs.output_trajectory
magnetic_moments = [magmom[0] for magmom in trajectory.get_array('atomic_magnetic_moments')]
print(magnetic_moments)

[2.3254]


In [11]:
from aiida_quantumespresso.utils.magnetic import generate_structure_with_magmoms

new_structure = generate_structure_with_magmoms(magnetic_structure, magnetic_moments)
print(f"New StructureData: {new_structure}\nAttached magmoms: {new_structure.properties.magmoms}")

New StructureData: uuid: e76003c3-7441-4d2b-85cf-699a87454f2f (pk: 1971)
Attached magmoms: [[0.0, 0.0, 2.3254]]


Using this utility function, kinds will be automatically detected. 

If you don't want the provenance and the new `StructureData` node to be stored, add `metadata={"store_provenance": False}` to the input of the `calcfunction`. 

### Non-collinear case 

For the non-collinear case, we just need to put `noncolin: True` in the input parameters:

In [12]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"noncolin": True}
            }
        }
    }
    )

In [13]:
run = run_get_node(builder)

09/26/2024 04:34:40 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1977|PwBaseWorkChain|run_process]: launching PwCalculation<1982> iteration #1
09/26/2024 04:34:59 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1977|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/26/2024 04:34:59 PM <1264724> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1977|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [14]:
run[0]

{'remote_folder': <RemoteData: uuid: 24c30625-332b-4ca8-aa94-02c08417efc4 (pk: 1983)>,
 'retrieved': <FolderData: uuid: 357ad2bd-63b1-4ed2-8b82-96b7e52a04b0 (pk: 1984)>,
 'output_parameters': <Dict: uuid: 9392a436-08e2-41ed-8845-22dd1c1d60cb (pk: 1987)>,
 'output_trajectory': <TrajectoryData: uuid: a3031313-41cc-4790-a6da-c69766ee5add (pk: 1986)>,
 'output_band': <BandsData: uuid: c9dfec73-e106-4a37-b412-4e84a0c648b7 (pk: 1985)>}

Currently, no parser for non-collinear magnetization is implemented in the `aiida-quantumespresso` plugin.

## How to run DFT+U (+V)

In the following we show how to run a DFT+U calculation. We consider the Hubbard parameters to be known (we don't compute them here).
As implemented in the [`HubbardStructureData`](https://aiida-vibroscopy.readthedocs.io/en/latest/5_iraman_functionals.html#defining-the-hubbardstructuredata), also for our atomistic `StructureData` we need to define the `Hubbard` class, now part of the `properties` attribute. We load the structure using pymatgen:

In [3]:
from pymatgen.core import Structure

Mn3Sn = Structure.from_file('../examples/structure/data/0.200_Mn3Sn.mcif', primitive=True)
mutable_structure = StructureDataMutable.from_pymatgen(Mn3Sn)

/opt/conda/lib/python3.10/site-packages/pymatgen/io/cif.py:1225: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [4]:
mutable_structure.properties.sites

[<SiteImmutable: kind name 'Mn0' @ 2.8325,3.3243285790685615,1.1327500000000004>,
 <SiteImmutable: kind name 'Mn1' @ 4.295203,0.7908526666851419,1.1327500000000004>,
 <SiteImmutable: kind name 'Mn0' @ 1.4632917100243504e-16,1.5817053333702837,3.39825>,
 <SiteImmutable: kind name 'Mn1' @ -1.4627029999999996,4.115181245753703,3.39825>,
 <SiteImmutable: kind name 'Mn4' @ 1.3697970000000002,0.7908526666851419,1.1327500000000001>,
 <SiteImmutable: kind name 'Mn4' @ 1.4627030000000005,4.115181245753703,3.3982500000000004>,
 <SiteImmutable: kind name 'Sn6' @ 2.528532938583794e-16,3.27068927495923,1.1327500000000004>,
 <SiteImmutable: kind name 'Sn6' @ 2.8325000000000005,1.6353446374796152,3.39825>]

This is a magnetic structure. However, let's suppose we don't want to run a magnetic calculation. We can remove the `magmoms` property using the 

In [5]:
mutable_structure.clear_property("magmoms")

mutable_structure.properties.magmoms

[[0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0]]

In [6]:
mutable_structure.get_defined_properties()

{'cell', 'kinds', 'masses', 'positions', 'symbols'}

Then, we attach the hubbard parameters using the `initialize_onsites_hubbard` method of the `StructureDataMutable`:

In [13]:
for kind in set(mutable_structure.properties.kinds):
    if "Mn" in kind:
        mutable_structure.initialize_onsites_hubbard(kind, '3d', 4, 'U', use_kinds=True)
mutable_structure.properties.hubbard.parameters

[HubbardParameters(atom_index=0, atom_manifold='3d', neighbour_index=0, neighbour_manifold='3d', translation=(0, 0, 0), value=4.0, hubbard_type='U'),
 HubbardParameters(atom_index=4, atom_manifold='3d', neighbour_index=4, neighbour_manifold='3d', translation=(0, 0, 0), value=4.0, hubbard_type='U'),
 HubbardParameters(atom_index=1, atom_manifold='3d', neighbour_index=1, neighbour_manifold='3d', translation=(0, 0, 0), value=4.0, hubbard_type='U')]

Please note that we have three Mn kinds, detected during the initialization of the structure (being magnetic, the three kinds are detected using the initial magmoms). We can change the kinds to coincide, so that we have only one Mn kind: `mutable_structure.clear_property("kinds")`.
We are now ready to run the calculation:

In [14]:
from aiida.engine import submit, run_get_node

structure = mutable_structure.to_immutable()

builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/30/2024 09:57:01 AM <1772356> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [2010|PwBaseWorkChain|run_process]: launching PwCalculation<2015> iteration #1
09/30/2024 10:00:19 AM <1772356> aiida.parser.PwParser: [ERROR] ERROR_ELECTRONIC_CONVERGENCE_NOT_REACHED
09/30/2024 10:00:19 AM <1772356> aiida.parser.PwParser: [ERROR] The electronic minimization cycle did not reach self-consistency.
09/30/2024 10:00:19 AM <1772356> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [WARNING] output parser returned exit code<410>: The electronic minimization cycle did not reach self-consistency.
09/30/2024 10:00:19 AM <1772356> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [2010|PwBaseWorkChain|report_error_handled]: PwCalculation<2015> failed with exit status 410: The electronic minimization cycle did not reach self-consistency.
09/30/2024 10:00:19 AM <1772356> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [2010|PwBaseWorkChain